In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import datapane as dp


ModuleNotFoundError: No module named 'numpy'

In [2]:
!pip3 install numpy pandas matplotlib seaborn plotly cufflinks statsmodels datapane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.3/253.3 kB 9.6 MB/s eta 0:00:00
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 820.8/820.

In [ ]:
import streamlit as st

In [ ]:
df = pd.read_excel("/Users/paniket/TU_Eindhoven/2_Study/Q2_JBI100_Visualisation_4/4_Code/JBI100_Visualisation/JBI100_Data_2024_2025/Australian_Shark_Incidents/Australian Shark-Incident Database Public Version.xlsx", sheet_name="ASID")

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
# Streamlit Title and Filters
st.title("Interactive Shark Attack Data Explorer")
st.sidebar.header("Filter Options")

# Filter by Year
year_range = st.sidebar.slider(
    "Select Year Range",
    int(df["Incident.year"].min()),
    int(df["Incident.year"].max()),
    (1900, 2023)
)
filtered_data = df[(df["Incident.year"] >= year_range[0]) & (df["Incident.year"] <= year_range[1])]

# Filter by State
state_filter = st.sidebar.multiselect(
    "Select State(s)",
    options=df["State"].unique(),
    default=df["State"].unique()
)
filtered_data = filtered_data[filtered_data["State"].isin(state_filter)]

In [ ]:
fig1 = px.line(
    filtered_data.groupby("Incident.year").size().reset_index(name="Count"),
    x="Incident.year",
    y="Count",
    title="Shark Attacks Over Time",
    labels={"Incident.year": "Year", "Count": "Number of Incidents"}
)
st.plotly_chart(fig1)


In [ ]:
fig2 = px.histogram(
    filtered_data,
    x="Site.category",
    color="Injury.severity",
    title="Attack Severity by Site Category",
    labels={"Site.category": "Site Category", "Injury.severity": "Severity"}
)
st.plotly_chart(fig2)


In [ ]:
# Check for NaN and invalid values
print(filtered_data["Shark.length.m"].isnull().sum())  # Count NaN values
print(filtered_data["Shark.length.m"].unique())  # Check unique values


In [ ]:
# Convert to numeric and handle errors
filtered_data["Shark.length.m"] = pd.to_numeric(filtered_data["Shark.length.m"], errors="coerce")

# Replace NaN values with a default size, such as the mean or a small default value
filtered_data["Shark.length.m"] = filtered_data["Shark.length.m"].fillna(1)  # Replace NaN with 1

# Alternatively, drop rows with NaN in Shark.length.m
# filtered_data = filtered_data.dropna(subset=["Shark.length.m"])


In [ ]:
filtered_data["Shark.length.m"].unique()[-1]

In [ ]:
fig3 = px.scatter_geo(
    filtered_data,
    lat="Latitude",
    lon="Longitude",
    color="Injury.severity",
    size="Shark.length.m",
    title="Geographic Distribution of Shark Attacks",
    labels={"Latitude": "Latitude", "Longitude": "Longitude"},
    projection="natural earth"
)

fig3.show()

In [ ]:
fig4 = px.bar(
    filtered_data,
    x="Victim.activity",
    color="Provoked/unprovoked",
    title="Victim Activities and Provocation",
    labels={"Victim.activity": "Activity", "Provoked/unprovoked": "Provocation Type"}
)
st.plotly_chart(fig4)


In [ ]:
fig5 = px.bar(
    filtered_data,
    x="Shark.common.name",
    color="Injury.severity",
    title="Shark Species Involved in Incidents",
    labels={"Shark.common.name": "Shark Species", "Injury.severity": "Severity"}
)
st.plotly_chart(fig5)


In [ ]:
# Graph Selection
graph_choice = st.sidebar.selectbox(
    "Select a Graph to Display",
    options=["Shark Attacks Over Time", "Attack Severity by Site Category", 
             "Geographic Distribution", "Victim Activity and Provocation", "Shark Species Involved"]
)

# Display the chosen graph
if graph_choice == "Shark Attacks Over Time":
    st.plotly_chart(fig1)
elif graph_choice == "Attack Severity by Site Category":
    st.plotly_chart(fig2)
elif graph_choice == "Geographic Distribution":
    st.plotly_chart(fig3)
elif graph_choice == "Victim Activity and Provocation":
    st.plotly_chart(fig4)
elif graph_choice == "Shark Species Involved":
    st.plotly_chart(fig5)


In [ ]:
streamlit run shark_attacks_app.py
